# Case | Expansão de uma rede de laboratórios

#### Objetivo:
*Apresentar uma análise exploratória dos dados de uma rede de medicina diagnóstica para definir caminhos de abordagem  que auxiliem a área de negócios da empresa a ter insumos iniciais para suportar seu plano de expansão.*

- Focar na fidelização de clientes (definir um nicho - talvez fazer um modelo de clusterização) para que eles retornem à rede quando surgir necessidade, visto que é difícil gerar demanda neste setor (mas ao mesmo tempo é um setor que nunca está em crise porque saúde sempre tem demanda - buscar informações que comprovem esta afirmação)
- Observar as localizações que a rede tem maior sucesso e menor sucesso
- Ver onde os clientes gastam mais
- Definir estratégias para melhorar a rentabilidade nas localizações que não fazem tanto sucesso? não é bem este o foco
- Seria melhor expandir onde tem menos sucesso para ver se a recepção melhora ou continuar abrindo onde faz mais sucesso?
- Seria melhor ir pelo caminho das intermediárias? O que os dados podem dizer sobre isso?

#### Após a análise
- apresentar e justificar a escolha de três zip codes para a instalação dos novos laboratórios da rede
- justificativas baseadas nos dados fornecidos e em considerações de negócio
- expor os resultados em uma apresentação clara e objetiva, incluindo todas as informações  e comentários relevantes
- como funciona o sistema de zip codes nos estados unidos?
- como é o mercado da medicina diagnóstica nos estados unidos?
- em que estado ou região há maior incidência de problemas de saúde (hipertensão, obesidade, onde há mais idosos ou maior natalidade)
- em que estado as pessoas costumam se preocupar mais com a saúde?
- como funciona o site / aplicativo / sistema de marcação de exames?
- geração de demandas: pacotes de check-up regular (não fazer exames só quando tem alguma doença ou outro problema de saúde)

#### etapas



#### importando as bibliotecas necessárias

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

#### ler as bases de dados

In [25]:
demographic_data = pd.read_csv('dados_brutos/DemographicData_ZCTAs.csv', index_col = 0)
df_geocode = pd.read_csv('dados_brutos/df_geocode.csv', index_col = 0)
economic_data = pd.read_csv('dados_brutos/EconomicData_ZCTAs.csv', index_col = 0)
test_data = pd.read_csv('dados_brutos/test_data.csv')
transactional_data = pd.read_csv('dados_brutos/transactional_data.csv', sep = ';')

#### qual o número de linhas e de colunas de cada base?

In [13]:
pd.DataFrame({
    'base': ['demographic_data', 'df_geocode', 'economic_data', 'test_data', 'transactional_data'],
    'n_colunas': [len(demographic_data.columns), len(df_geocode.columns), len(economic_data.columns), len(test_data.columns), len(transactional_data.columns)],
    'n_linhas': [len(demographic_data), len(df_geocode), len(economic_data), len(test_data), len(transactional_data)]
})

,base,n_colunas,n_linhas
0,demographic_data,18,33120
1,df_geocode,4,119
2,economic_data,12,629280
3,test_data,6,2002
4,transactional_data,8,2355241


#### verificando as primeiras linhas de cada dataframe

1. demographic_data

In [12]:
# dados demográficos relevantes a nível de ZCTA (zip code tabulation area)
demographic_data.head()

,Id,GeographicAreaName,TotalPopulation,SexRatio(males per 100 females),Population_Under5Years,Population_5to9Years,Population_10to14Years,Population_15to19Years,Population_20to24Years,Population_25to34Years,Population_35to44Years,Population_45to54Years,Population_55to59Years,Population_60to64Years,Population_65to74Years,Population_75to84Years,Population_85YearsAndOver,MedianAgeInYears
0,8600000US35004,ZCTA5 35004,12045,94.1,805,1075,898,477,578,2088,1628,1200,886,683,1017,534,176,35.5
1,8600000US35005,ZCTA5 35005,7344,86.1,504,453,511,499,214,788,795,968,612,561,798,485,156,44.0
2,8600000US35006,ZCTA5 35006,2883,108.2,96,153,303,129,156,183,367,430,296,260,280,201,29,47.2
3,8600000US35007,ZCTA5 35007,26332,95.0,1936,1992,1837,1762,1376,3119,3849,3907,1665,1323,2096,1106,364,37.7
4,8600000US35010,ZCTA5 35010,20613,90.5,1306,1465,944,1217,1128,2513,2106,2950,1512,1472,2421,1155,424,42.6


2. df_geocode

In [11]:
# informações a respeito da localização de cada laboratório, como endereço, lat-long e ZCTA (zipcode) no qual o laboratório se encontra
df_geocode.head()

,Lab Id,Address,Location,Zipcode
0,L152,"3800 PLEASANT HILL RD STE 1, DULUTH, GA 30096","34.000319,-84.1629724",30096.0
1,L520,"1614 N JAMES ST, ROME, NY 13440","43.2311327,-75.4445363",13440.0
2,L141,"12911 120TH AVE NE STE D60, KIRKLAND, WA 98034","47.7162786,-122.1838152",98034.0
3,L524,"5667 PEACHTREE DUNWOODY RD 250, ATLANTA, GA 30342","33.9093875,-84.3529096",30342.0
4,L545,"1204 IL HWY 164, OQUAWKA, IL 61469","40.9309925,-90.9437598",61469.0


3. economic_data

In [26]:
# dados econômicos relevantes a nível de ZCTAs
economic_data.head()

,id,Geographic Area Name,TotalHouseholds_LessThan$10.000,TotalHouseholds_$10.000to$14.999,TotalHouseholds_$15.000to$24.999,TotalHouseholds_$25.000to$34.999,TotalHouseholds_$35.000to$49.999,TotalHouseholds_$50.000to$74.999,TotalHouseholds_$75.000to$99.999,TotalHouseholds_$100.000to$149.999,TotalHouseholds_$150.000to$199.999,TotalHouseholds_$200.000OrMore
0,8600000US35004,ZCTA5 35004,198,71,298,513,647,1117,529,945,245,61
1,8600000US35005,ZCTA5 35005,188,184,318,293,353,562,299,407,67,26
2,8600000US35006,ZCTA5 35006,71,20,117,104,154,176,124,194,51,7
3,8600000US35007,ZCTA5 35007,396,208,670,462,1173,1854,1578,2224,473,254
4,8600000US35010,ZCTA5 35010,700,610,1093,957,1056,1512,807,749,254,249


In [27]:
# a coluna "id" está em minúsculo. importante alterar para "Id", visto que no dataframe "demographic_data" o nome da coluna está desta forma

economic_data = economic_data.rename({'id':'Id'}, axis = 1)
economic_data.head()

,Id,Geographic Area Name,TotalHouseholds_LessThan$10.000,TotalHouseholds_$10.000to$14.999,TotalHouseholds_$15.000to$24.999,TotalHouseholds_$25.000to$34.999,TotalHouseholds_$35.000to$49.999,TotalHouseholds_$50.000to$74.999,TotalHouseholds_$75.000to$99.999,TotalHouseholds_$100.000to$149.999,TotalHouseholds_$150.000to$199.999,TotalHouseholds_$200.000OrMore
0,8600000US35004,ZCTA5 35004,198,71,298,513,647,1117,529,945,245,61
1,8600000US35005,ZCTA5 35005,188,184,318,293,353,562,299,407,67,26
2,8600000US35006,ZCTA5 35006,71,20,117,104,154,176,124,194,51,7
3,8600000US35007,ZCTA5 35007,396,208,670,462,1173,1854,1578,2224,473,254
4,8600000US35010,ZCTA5 35010,700,610,1093,957,1056,1512,807,749,254,249


4. test_data

In [11]:
# informações a respeito dos exames, características como custo de aplicação e outras especificidades técnicas
test_data.head()

,CodItem,Desc Item,Category,Family,Speciality,Testing Cost
0,70003237,"MAGNESIO, SORO (EXEMPLO DE EXPRESSAO DE RESULT...",CA,CORELAB,BIOCHEMISTRY,1.78
1,70000638,"HEMOGRAMA, AUTOMATIZADO, SANGUE",CA,HEMATO,BLOOD COUNT,2.46
2,70001597,"FERRITINA, SORO",CA,CORELAB,IMMUNOHORMONE,2.11
3,70000103,"FERRO, DOSAGEM, SORO (EXEMPLO DE EXPRESSAO DE ...",CA,CORELAB,BIOCHEMISTRY,0.80
4,70000224,"CALCIO, DOSAGEM, SORO (EXEMPLO DE EXPRESSAO DE...",CA,CORELAB,BIOCHEMISTRY,1.02


5. transactional_data

In [7]:
# registros de exames feitos por pacientes da rede em cada laboratório
transactional_data.head()

,Patient Id,Gender,Date of birth,Date of service,Service Id,Lab Id,CodItem,Testing Cost
0,10210830256-BIO003,F,01/08/1976 00:00:00,2019-01-07,571904533475-38,L133,70003237,"9,0"
1,10210830256-BIO003,F,01/08/1976 00:00:00,2019-01-07,571904533475-38,L133,70000638,"13,0"
2,10210830256-BIO003,F,01/08/1976 00:00:00,2019-01-07,571904533475-38,L133,70001597,"49,0"
3,10210830256-BIO003,F,01/08/1976 00:00:00,2019-01-07,571904533475-38,L133,70000103,"11,0"
4,10210830256-BIO003,F,01/08/1976 00:00:00,2019-01-07,571904533475-38,L133,70000224,"10,0"


#### mapeamento de colunas de id (que podem relacionar uma base com outra)

criando um dataframe com a lista de colunas de cada base

In [88]:
df1 = demographic_data.columns.tolist()
df1 = pd.DataFrame(df1, columns = ['columns'])
df1['1_demographic_data'] = 'x'

df2 = df_geocode.columns.tolist()
df2 = pd.DataFrame(df2, columns = ['columns'])
df2['2_df_geocode'] = 'x'

df3 = economic_data.columns.tolist()
df3 = pd.DataFrame(df3, columns = ['columns'])
df3['3_economic_data'] = 'x'

df4 = test_data.columns.tolist()
df4 = pd.DataFrame(df4, columns = ['columns'])
df4['4_test_data'] = 'x'

df5 = transactional_data.columns.tolist()
df5 = pd.DataFrame(df5, columns = ['columns'])
df5['5_transactional_data'] = 'x'

# exemplo de como fica o dataframe com a lista de colunas para cada base:
df2


,columns,2_df_geocode
0,Lab Id,x
1,Address,x
2,Location,x
3,Zipcode,x


juntando todas as listas criadas para verificar as colunas em comum

In [73]:
merged = df1.merge(df2, how = 'outer').merge(df3, how = 'outer').merge(df4, how = 'outer').merge(df5, how = 'outer')
merged

,columns,1_demographic_data,2_df_geocode,3_economic_data,4_test_data,5_transactional_data
0,Id,x,NaN,x,NaN,NaN
1,GeographicAreaName,x,NaN,NaN,NaN,NaN
2,TotalPopulation,x,NaN,NaN,NaN,NaN
3,SexRatio(males per 100 females),x,NaN,NaN,NaN,NaN
4,Population_Under5Years,x,NaN,NaN,NaN,NaN
5,Population_5to9Years,x,NaN,NaN,NaN,NaN
6,Population_10to14Years,x,NaN,NaN,NaN,NaN
7,Population_15to19Years,x,NaN,NaN,NaN,NaN
8,Population_20to24Years,x,NaN,NaN,NaN,NaN
9,Population_25to34Years,x,NaN,NaN,NaN,NaN


filtrando apenas as colunas que aparecem em pelo menos 2 bases

In [87]:
merged['n_dfs'] = merged.count(axis = 1) - 2 # conta os número de colunas com valores não-nulos, subtraindo 2 (index e coluna dos nomes)
merged[merged['n_dfs'] >= 2] # filtra apenas os nomes de colunas que aparecem em pelo menos 2 bases

,columns,1_demographic_data,2_df_geocode,3_economic_data,4_test_data,5_transactional_data,n_dfs
0,Id,x,NaN,x,NaN,NaN,2
18,Lab Id,NaN,x,NaN,NaN,x,2
33,CodItem,NaN,NaN,NaN,x,x,2
38,Testing Cost,NaN,NaN,NaN,x,x,2


Observando o dataframe acima, é possível perceber que as bases test_data e transactional_data possuem 2 colunas em comum. É importante investigar se essas duas colunas são correspondentes